<a href="https://colab.research.google.com/github/hagigat/NeuralNetwork/blob/master/RBF_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib as plt
import math
import scipy.spatial.distance as sc
from sklearn.metrics import accuracy_score

Read Data From pima_indians_diabetes Dataset

In [0]:
data = pd.read_csv("pima_indians_diabetes.csv", header=None)
data = data.dropna().to_numpy()
X_train = data[0:650, 0:8]
Y_train = data[0:650, 8]
X_test = data[650:760, 0:8]
Y_test = data[650:760, 8]

In [0]:
class RBF:
    def __init__(self, X, Y):
        self.nl = 0
        self.X = self.normalize(X)
        self.Y = Y
        self.C = np.zeros((0, 8))
        self.weight = []
        self.std = self.deviation()
        self.nlmax = int(300)

    def deviation(self):
        return np.amax(sc.pdist(self.X, 'euclidean')) / np.sqrt(2 * self.X.shape[1])

    def Gaussian(self):
        ed = sc.cdist(self.X, self.C, 'euclidean')
        return np.exp(-0.5 * ((ed / self.std) ** 2))

    def tGaussian(self,X):
        ed = sc.cdist(X, self.C, 'euclidean')
        return np.exp(-0.5 * ((ed / self.std) ** 2))    

    def w_opt(self, O, D):
        tmp = np.linalg.inv(np.dot(O.T, O))
        self.weight = np.dot(tmp, np.dot(O.T, D))

    def normalize(self, X):
        for i in range(X.shape[1]):
            X[:, i] = (X[:, i] - X[:, i].mean()) / np.std(X[:, i])
        return X

    def Train(self):
        # RBF Train
        Xt = self.X.copy()
        for i in range(self.nlmax):
            self.nl += 1
            idx = np.random.randint(0, (self.X.shape[0] -1) - i)
            self.C = np.append(self.C, [Xt[idx]], axis=0)
            Xt = np.delete(Xt, idx, 0)
            O = self.Gaussian()
            O = np.c_[O, np.ones(len(self.X))]
            self.w_opt(O, self.Y)
            y = self.act_fun(np.dot(O, self.weight))
            mse = np.sum((y-self.Y)**2) / self.X.shape[0]
            print("NL = ",self.nl, "-----> MSE :","%.4f" % mse," , Accuracy: ","%.1f" % (accuracy_score(self.Y, y)*100))
            if mse < 0.08:
                break
        print("nl : ", self.nl)
        print("C : ", self.C.shape)
        print("W : ", self.weight.shape)    

    def Test(self, X, Y):
        # RBF Test
        O = self.tGaussian(X)
        O = np.c_[O, np.ones(len(X))]
        y = self.act_fun(np.dot(O, self.weight))
        mse = np.sum((y-Y)**2) / X.shape[0]
        print("> MSE :","%.4f" % mse," , Accuracy: ","%.1f" % (accuracy_score(Y, y)*100))


    def act_fun(self, z):  # sigmoid
        result = 1.0 / (1.0 + np.exp(-z))
        result[result >= 0.5] = 1
        result[result < 0.5] = 0
        return result

    def relu(self, z):
        if np.isscalar(z):
            result = np.max((z, 0))
        else:
            zero_aux = np.zeros(z.shape)
            meta_z = np.stack((z, zero_aux), axis=-1)
            result = np.max(meta_z, axis=-1)
            return result

Define Instance of RBF Class

In [0]:
rbf = RBF(X_train, Y_train)
rbf.Train()

NL =  1 -----> MSE : 0.6569  , Accuracy:  34.3
NL =  2 -----> MSE : 0.6431  , Accuracy:  35.7
NL =  3 -----> MSE : 0.5462  , Accuracy:  45.4
NL =  4 -----> MSE : 0.5462  , Accuracy:  45.4
NL =  5 -----> MSE : 0.5554  , Accuracy:  44.5
NL =  6 -----> MSE : 0.5477  , Accuracy:  45.2
NL =  7 -----> MSE : 0.5508  , Accuracy:  44.9
NL =  8 -----> MSE : 0.5600  , Accuracy:  44.0
NL =  9 -----> MSE : 0.5508  , Accuracy:  44.9
NL =  10 -----> MSE : 0.5554  , Accuracy:  44.5
NL =  11 -----> MSE : 0.5477  , Accuracy:  45.2
NL =  12 -----> MSE : 0.5446  , Accuracy:  45.5
NL =  13 -----> MSE : 0.5446  , Accuracy:  45.5
NL =  14 -----> MSE : 0.5477  , Accuracy:  45.2
NL =  15 -----> MSE : 0.5569  , Accuracy:  44.3
NL =  16 -----> MSE : 0.5646  , Accuracy:  43.5
NL =  17 -----> MSE : 0.5600  , Accuracy:  44.0
NL =  18 -----> MSE : 0.5615  , Accuracy:  43.8
NL =  19 -----> MSE : 0.5523  , Accuracy:  44.8
NL =  20 -----> MSE : 0.5508  , Accuracy:  44.9
NL =  21 -----> MSE : 0.5431  , Accuracy:  45.7
N

In [0]:
X_test = rbf.normalize(X_test)
rbf.Test(X_test,Y_test)

> MSE : 0.4455  , Accuracy:  55.5
